<a href="https://colab.research.google.com/github/francisan17/CuPractice/blob/main/c(2x2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
import pandas as pd
from tabulate import tabulate
from cu2o_bulk import cu2o_bulk, cu2o100, Oterm1x1, c2x2

bulk = cu2o_bulk()
bulk.calc = mace_mp(model="small", dispersion=True, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()
bulk_oxygens = bulk[bulk.symbols=='O']
print(f"Number of Oxygen atoms in bulk: {len(bulk_oxygens)}")
#write('bulk.xyz', bulk)

#n_layers=3
#vacuum=10
#slab = cu2o100(bulk, n_layers, vacuum)
#view(slab)

n_layers = { 3,4,5,6,7,8,9,10}
Cu2Otable={}

n_layers_list=[]
slab_oxygens_list=[]
E_surf_list=[]


for n_layers in range(3,11):
  vacuum=10
  slab = c2x2(bulk, n_layers, vacuum)
  #write('slab.xyz', slab)
  slab_oxygens = slab[slab.symbols=='O']
  Cu2Otable[n_layers]=len(slab_oxygens)
  print(f"Number of Oxygen atoms in slab: {len(slab_oxygens)}")

  slab.calc = mace_mp(model="small", dispersion=True, default_dtype="float64", device='cpu')
  E_slab = slab.get_potential_energy()
  E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_cleav=}')

  qn = LBFGS(slab, trajectory='c2x2.traj')
  qn.run(fmax=0.01)
  E_slab = slab.get_potential_energy()

  ##Calc Surface Energy
  E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_surf=}')

  n_layers_list.append(n_layers)
  slab_oxygens_list.append(len(slab_oxygens))
  E_surf_list.append(E_surf)

df = pd.DataFrame({'Number of Layers': n_layers_list, 'Number of Oxygen Atoms':slab_oxygens_list, 'Surface Energy': E_surf_list})
table=tabulate(df, headers = 'keys', tablefmt = 'fancy_grid')
df.to_csv('c2x2.csv', index=False)
print(table)

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections (see https://github.com/pfnet-research/torch-dftd)
       Step     Time          Energy         fmax
LBFGS:    0 14:46:15      -30.743969        0.2829


/home/lana/ve/lib64/python3.11/site-packages/torch_dftd/torch_dftd3_calculator.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  cell: Optional[Tensor] = torch.tensor(


LBFGS:    1 14:46:15      -30.747281        0.2635
LBFGS:    2 14:46:15      -30.768450        0.0140
LBFGS:    3 14:46:16      -30.768506        0.0006
Number of Oxygen atoms in bulk: 2
Number of Oxygen atoms in slab: 8
Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/46jrkm3v
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections (see https://github.com/pfnet-research/torch-dftd)
n_layers=3 E_cleav=-0.3070382289812538
       Step     Time          Energy         fmax
LBFGS:    0 14:46:16     -114.385257        1.7329
LBFGS:    1 14:46:17     -114.640391        1.4163
LBFGS:    2 14:46:17     -115.060751        0.9692
LBFGS:    3 14:46:17     -115.243716        1.0024
LBFGS:    4 14:46:18     -115.591388        0.9887
LBFGS:    5 14:46:18     -115.709531        0.8856
LBFGS:    6 14:46:18     -115.954040        0.7789
LBFGS:    7 14:46:18     -116.11

KeyboardInterrupt: 

In [1]:
!pip install torch-dftd

  Using cached torch_dftd-0.4.0-py3-none-any.whl (681 kB)
  Using cached ase-3.23.0-py3-none-any.whl (2.9 MB)
  Using cached pymatgen-2024.6.10-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.2/809.2 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7